# EXPLORATION DES DONNÉES DES POLLUANTS

## Objectifs :

- Extraire la liste des polluants recherchés dans les prélèvements
- Sortir des premières analyses de ces données
- Disponibilité de la donnée
- Couverture du territoire : pour chaque polluant, quelle est la part des UDI avec analyse de chaque polluant
- Périodicité : pour chaque polluant, à quelle fréquence collecte-t-on ?
- Visualisation sur carte de la donnée pour chaque polluant. (obj : avoir une idée de la distribution, des valeurs min max etc)
- Analyse des polluants par seuils de dépassement

<p> => Analyser la donnée pour chaque polluant</p><br> 

---

<br>
<br>

> # Première étape : comprendre la donnée et les tables disponibles

In [2]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données et par connecter la base de données DuckDB

import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [3]:
# Liste des différentes tables présentes dans la base de données

tables = con.execute("SHOW TABLES").fetch_df()
tables

,name
0,edc_communes
1,edc_prelevements
2,edc_resultats


In [4]:
# Comprendre la data dans chaque table

# Table edc_communes
con.execute("DESCRIBE edc_communes").fetchdf()

,column_name,column_type,null,key,default,extra
0,inseecommune,VARCHAR,YES,None,None,None
1,nomcommune,VARCHAR,YES,None,None,None
2,quartier,VARCHAR,YES,None,None,None
3,cdreseau,VARCHAR,YES,None,None,None
4,nomreseau,VARCHAR,YES,None,None,None
5,debutalim,DATE,YES,None,None,None
6,de_partition,INTEGER,YES,None,None,None
7,de_ingestion_date,DATE,YES,None,None,None


In [5]:
# Table edc_prelevements

con.execute("DESCRIBE edc_prelevements").fetchdf()

,column_name,column_type,null,key,default,extra
0,cddept,VARCHAR,YES,None,None,None
1,cdreseau,VARCHAR,YES,None,None,None
2,inseecommuneprinc,VARCHAR,YES,None,None,None
3,nomcommuneprinc,VARCHAR,YES,None,None,None
4,cdreseauamont,VARCHAR,YES,None,None,None
5,nomreseauamont,VARCHAR,YES,None,None,None
6,pourcentdebit,VARCHAR,YES,None,None,None
7,referenceprel,VARCHAR,YES,None,None,None
8,dateprel,DATE,YES,None,None,None
9,heureprel,VARCHAR,YES,None,None,None


In [6]:
# Table edc_resultats

con.execute("DESCRIBE edc_resultats").fetchdf()

,column_name,column_type,null,key,default,extra
0,cddept,VARCHAR,YES,None,None,None
1,referenceprel,VARCHAR,YES,None,None,None
2,cdparametresiseeaux,VARCHAR,YES,None,None,None
3,cdparametre,BIGINT,YES,None,None,None
4,libmajparametre,VARCHAR,YES,None,None,None
5,libminparametre,VARCHAR,YES,None,None,None
6,libwebparametre,VARCHAR,YES,None,None,None
7,qualitparam,VARCHAR,YES,None,None,None
8,insituana,VARCHAR,YES,None,None,None
9,rqana,VARCHAR,YES,None,None,None


In [7]:
# Aperçu de la table "edc_communes"

edc_communes = con.execute("SELECT * FROM edc_communes LIMIT 5").fetch_df()
edc_communes

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2024,2025-02-07
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2024,2025-02-07
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2024,2025-02-07
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2024,2025-02-07
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2024,2025-02-07


In [8]:
# Aperçu de la table "edc_prelevements"

edc_prelevements = con.execute("SELECT * FROM edc_prelevements LIMIT 5").fetch_df()
edc_prelevements

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024,2025-02-07
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07
3,001,001000003,01007,AMBRONAY,None,None,None,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07
4,001,001000003,01007,AMBRONAY,None,None,None,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07


In [9]:
# Aperçu de la table "edc_resultats"

edc_resultats = con.execute("SELECT * FROM edc_resultats LIMIT 5").fetch_df()
edc_resultats

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date
0,001,00100138503,ALTMICR,1370,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,None,N,L,<10,µg/L,133,None,<=200 µg/L,0.00,7429-90-5,00100147233,2024,2025-02-07
1,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147232,2024,2025-02-07
2,001,00100138503,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147233,2024,2025-02-07
3,001,00100138503,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,N,T,"0,30",mg(Cl2)/L,165,None,None,0.30,None,None,2024,2025-02-07
4,001,00100138503,CL2TOT,1399,CHLORE TOTAL,Chlore total,None,N,T,"0,34",mg(Cl2)/L,165,None,None,0.34,None,None,2024,2025-02-07


> # Deuxième étape : réaliser des analyses simples pour comprendre les datasets

### 1 - Analyse de la table edc_communes

In [10]:
# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de communes

communes = con.table("edc_communes").to_df()
nombre_de_communes = communes.nunique()["inseecommune"]
print(f"nombre_de_communes = {nombre_de_communes}")

# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de réseaux uniques

reseau = con.table("edc_communes").to_df()
nb_reseaux_uniques = reseau.nunique()["cdreseau"]
print(f"nombre_de_reseaux = {nb_reseaux_uniques}")

nombre_de_communes = 34809
nombre_de_reseaux = 23184


In [11]:
# Combien de communes déservies par chaque réseau ?

query = """
SELECT cdreseau, nomreseau, COUNT(DISTINCT inseecommune) AS nb_communes
FROM edc_communes
GROUP BY cdreseau, nomreseau
ORDER BY nb_communes DESC;
"""

communes_par_reseau = con.execute(query).fetch_df()
communes_par_reseau

,cdreseau,nomreseau,nb_communes
0,032000434,BAROUSSE COMMINGES SAVE,79
1,059000481,EBBLINGHEM,72
2,031000096,MONTAGNE NOIRE,61
3,031000109,ST NERE TROUBAT CLARAC VILLENEUVE,61
4,057001421,SEBVF 2,57
...,...,...,...
23179,974000027,RESEAU MANAPANY LES HAUTS,1
23180,974000068,RESEAU LA MONTAGNE,1
23181,974000071,RESEAU J.PTI LES HTS GD COUDE,1
23182,974000119,RESEAU MONTEE SANO,1


In [57]:
# Combien de réseaux desservent > 10 communes ?

where_clause = "HAVING nb_communes > 10"

query = f"""
SELECT cdreseau, nomreseau, COUNT(DISTINCT inseecommune) AS nb_communes
FROM edc_communes
GROUP BY cdreseau, nomreseau
{where_clause}
ORDER BY nb_communes DESC;
"""

reseau_desservant_plus_de_10_communes = con.execute(query).fetch_df()
reseau_desservant_plus_de_10_communes

# % de réseaux desservant > 10 communes sur le total des réseaux

nb_reseaux_desservant_plus_de_10_communes = reseau_desservant_plus_de_10_communes.shape[
    0
]
pourcentage_reseaux_desservant_plus_de_10_communes = (
    nb_reseaux_desservant_plus_de_10_communes / nb_reseaux_uniques * 100
)
print(
    f"pourcentage reseaux desservant plus de 10 communes = {pourcentage_reseaux_desservant_plus_de_10_communes:.2f}%"
)

# Combien de réseaux desservent uniquement une seule commune ?

where_clause = "HAVING nb_communes = 1"

query = f"""
SELECT cdreseau, nomreseau, COUNT(DISTINCT inseecommune) AS nb_communes
FROM edc_communes
GROUP BY cdreseau, nomreseau
{where_clause}
ORDER BY nb_communes DESC;
"""

reseau_desservant_une_seule_commune = con.execute(query).fetch_df()
reseau_desservant_une_seule_commune

# % de réseaux desservant une seule commune sur le total des réseaux

nb_reseaux_desservant_une_seule_commune = reseau_desservant_une_seule_commune.shape[0]
pourcentage_reseaux_desservant_une_seule_commune = (
    nb_reseaux_desservant_une_seule_commune / nb_reseaux_uniques * 100
)
print(
    f"pourcentage reseaux desservant seulement 1 commune = {pourcentage_reseaux_desservant_une_seule_commune:.2f}%"
)


pourcentage reseaux desservant plus de 10 communes = 2.54%
pourcentage reseaux desservant seulement 1 commune = 76.85%


La grande majorité des réseaux déservent une commune unique (77%). Seulement 3% des réseaux desservent plus de 10 communes.


In [51]:
# Y a-t-il des communes desservies par plusieurs réseaux ?

query = """
SELECT inseecommune, COUNT(DISTINCT cdreseau) AS nb_reseaux
FROM edc_communes
GROUP BY inseecommune
HAVING nb_reseaux > 1
ORDER BY nb_reseaux DESC;
"""

communes_desservies_par_plusieurs_reseaux = con.execute(query).fetch_df()
communes_desservies_par_plusieurs_reseaux

,inseecommune,nb_reseaux
0,48027,19
1,73257,19
2,48009,18
3,73150,17
4,13055,16
...,...,...
6409,83131,2
6410,86222,2
6411,88276,2
6412,88388,2


La relation réseau <> commune est une relation many to many. Un réseau peut desservir plusieurs communes. Une commune peut être desservie par plusieurs réseaux. La majorité des communes ont un unique réseau qui les dessert.

- Comment faire pour la cartographie ? Si une commune a plusieurs réseaux et que la qualité de l'eau de ces derniers est hétérogène ?

-> Si un réseau mauvais alors toute la commune marquée à risque ?  
-> Faire la cartographie de manière plus précise que par communes ?

### 2 - Analyse de la table edc_prelevements

### 3 - Analyse de la table edc_resultats